In [3]:
import sys
import glob
from os.path import splitext, basename
from datetime import datetime

from delf import feature_io
from scipy.spatial import cKDTree
from skimage.measure import ransac

sys.path.append('/home/alexandrearaujo/library/faiss/')
import faiss

import numpy as np

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [ ]:
_DISTANCE_THRESHOLD = 0.8

In [41]:
query_files = glob.iglob('/media/hdd1/kaggle/landmark-retrieval-challenge/feature_test_256x256/*')
index_files = glob.iglob('/media/hdd1/kaggle/landmark-retrieval-challenge/feature_index_256x256/*')

In [42]:
results = {}
total_dim = 0
for i, path_query in enumerate(index_files):
    filename_query = splitext(basename(path_query))[0]
    if filename_query != '03215fbefac009f3': continue
    try:
        query_loc, _, query_desc, _, _ = feature_io.ReadFromFile(path_query)
    except Exception as error:
        print('problem with file {}. error : {}'.format(filename_query, error))
    break

In [ ]:
results = {}
min_dim = np.iinfo(np.int32).max
for i, path_index in enumerate(index_files):
    if i == 1000: print(i)
    if i == 10000: print(i)
    if i % 100000 == 0: print(i)
    filename = splitext(basename(path_index))[0]
    try:
        index_loc, _, index_desc, _, _ = feature_io.ReadFromFile(path_index)
    except:
        print(filename)
    min_dim = np.min([index_desc.shape[0], min_dim])
print(min_dim)

0
1000
10000
100000
f818bb32aad0a4a1
5f3e7289c8d8bacb
200000
6f5819e00cdd1acb
300000
400000
49a6372f1c83e0fb


In [ ]:
def mmap_fvecs(fname):
    x = np.memmap(fname, dtype='int32', mode='r')
    d = x[0]
    return x.view('float32').reshape(-1, d + 1)[:, 1:]

def mmap_bvecs(fname):
    x = np.memmap(fname, dtype='uint8', mode='r')
    d = x[:4].view('int32')[0]
    return x.reshape(-1, d + 4)[:, 4:]

In [21]:
def read_memmap_file(path, dtype='float32'):
    return np.memmap(path, dtype=dtype, mode='r')

In [35]:
def pickle_load(path):
    import pickle
    """function to load pickle object"""
    with open(path, 'rb') as f:
        return pickle.load(f, encoding='latin1')

In [22]:
a = read_memmap_file('../data/xdatabase_mean_agg.bin')

In [45]:
a[0]

memmap([-0.09381485, -0.28293085, -0.0871534 ,  0.24709901,  0.01396151,
         0.18320307,  0.0084712 ,  0.02998371,  0.01800802, -0.01926095,
         0.03163223, -0.08175791,  0.0300359 , -0.01396307, -0.00414801,
         0.01534101,  0.00370465, -0.00786319, -0.05978213,  0.08471166,
        -0.01892041,  0.01734073,  0.05149697, -0.04952597,  0.02972577,
        -0.01915675,  0.01722981,  0.06007157,  0.02012954,  0.0056765 ,
        -0.03618811, -0.01234898,  0.02470871, -0.03124979, -0.03923102,
         0.00722395, -0.07422312,  0.02352059,  0.02981085, -0.04042541],
       dtype=float32)

In [28]:
query_desc.mean(axis=0)

array([-0.11956213, -0.13958703,  0.1120486 , -0.02599497, -0.06480798,
        0.00081129,  0.08376851,  0.03655818, -0.12371404, -0.0297387 ,
        0.01222533,  0.04212876, -0.08973299,  0.02073307, -0.03001502,
       -0.05496655, -0.0054426 ,  0.01806564,  0.06029709, -0.0409293 ,
       -0.00667151, -0.00191101,  0.03286559, -0.01167688, -0.00056594,
        0.02953819, -0.11802831, -0.03426814, -0.02435733, -0.00892419,
       -0.00089785,  0.01862487, -0.0177884 , -0.01624243, -0.00896583,
        0.02121899, -0.03586563,  0.02604675,  0.00092811,  0.00724195])

In [36]:
pickle_load('../data/filename_index.bin')

array(['03215fbefac009f3', 'f1b588f156e94454', '1458547c8874189b', ...,
       '6297f36059d67883', 'bec0e4d68bfe491f', '3fd092bd56072896'],
      dtype='<U16')

In [44]:
query_desc.mean(axis=0)

array([-0.09381485, -0.28293085, -0.0871534 ,  0.24709902,  0.01396151,
        0.18320307,  0.0084712 ,  0.02998371,  0.01800802, -0.01926095,
        0.03163223, -0.08175791,  0.0300359 , -0.01396307, -0.00414801,
        0.01534101,  0.00370465, -0.00786319, -0.05978213,  0.08471166,
       -0.01892041,  0.01734073,  0.05149697, -0.04952597,  0.02972577,
       -0.01915675,  0.01722981,  0.06007157,  0.02012954,  0.0056765 ,
       -0.03618812, -0.01234898,  0.02470871, -0.03124979, -0.03923102,
        0.00722395, -0.07422311,  0.02352059,  0.02981085, -0.04042541])

In [ ]:
# In Python, a product quantizer is defined by:

d = 
m = 5                                    # number of subquantizers
n_bits = 10                              # bits allocated per subquantizer
pq = faiss.IndexPQ (d, m, n_bits)        # Create the index
pq.train (x_train)                       # Training
pq.add (x_base)                          # Populate the index
D, I = pq.search (x_query, k)            # Perform a search

# The number of bits n_bits must be equal to 8, 12 or 16. The dimension d should be a multiple of m